In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/gemstone.csv', nrows=25000)
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df = df.drop(labels=['id'],axis=1)

In [4]:
## Independent and dependent features

X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [5]:
Y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
24995,11950
24996,9032
24997,7094
24998,1959


In [6]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
24995,1.51,Ideal,F,SI1,62.3,56.0,7.34,7.40,4.60
24996,1.03,Ideal,F,VVS2,61.3,56.0,6.55,6.53,4.01
24997,1.02,Premium,E,VS1,62.7,58.0,6.42,6.34,4.00
24998,0.71,Premium,J,SI1,60.3,60.0,5.74,5.68,3.45


In [7]:
## Categorical columns(ordinal) and numerical columns

categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [8]:
## Custom ranking of each ordinal variables

cut_categories = ['Fair','Good','Very Good','Premium','Ideal']
color_categories = ['D','E','F','G','H','I','J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [9]:
from sklearn.impute import SimpleImputer  ## for handling missing values
from sklearn.preprocessing import StandardScaler ## For feature Scaling
from sklearn.preprocessing import OrdinalEncoder ## for ordinal encoding

## Pipelines

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [10]:
## Numerical Pipeline

num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

## Categorical Pipeline

cat_pipeline = Pipeline(
    steps=[
        ('impute',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [11]:
## Train Test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.30,random_state=42)

In [12]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [13]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.446453,-1.111014,1.437049,0.594385,0.638883,0.515532,-1.141534,0.912693,-0.659483
1,-0.176006,-0.467677,-0.643808,0.065857,0.025818,-0.006050,0.871652,-1.550431,0.004859
2,0.575238,-1.294825,0.396620,0.764589,0.801165,0.616951,-0.134941,-0.934650,-0.659483
3,-0.991641,-0.651488,-0.643808,-1.134527,-1.101139,-1.165120,-1.141534,0.296912,1.333543
4,-0.111613,0.083755,-1.164023,0.056899,0.097943,0.109858,0.871652,-0.934650,-1.323824


In [14]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [15]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [16]:
regression.coef_

array([[ 6579.6568362 ,   -94.90887622,   -57.74503512, -1752.1895584 ,
         -193.5199485 ,  -436.39483685,    70.1238781 ,  -464.87046528,
          645.60757895]])

In [17]:
regression.intercept_

array([4000.2216])

In [18]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [19]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1063.8435830777726
MAE: 687.6008019338491
R2 score 93.24754151665041


Lasso
Model Training Performance
RMSE: 1063.1296374169622
MAE: 688.3495119641013
R2 score 93.25660162823263


Ridge
Model Training Performance
RMSE: 1063.6737215425715
MAE: 687.7560496443384
R2 score 93.24969764451451


Elasticnet
Model Training Performance
RMSE: 1537.839059513325
MAE: 1067.917296109121
R2 score 85.88996338722829




In [20]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']